# Version 1 of the final project 

## Preprocessing 

In [12]:
#Imports necessary libraries 
import pandas as pd
import numpy as np
import networkx as nx
import json as js
import requests
import os

In [8]:
data = js.load(open('tweet_jsons/response1.json'))
df = pd.DataFrame(data["data"])

for i in range(2, 63):
    filepath = f'tweet_jsons/response{i}.json'
    data = js.load(open(filepath))
    new_df = pd.DataFrame(data["data"])
    df = pd.concat([new_df, df])

print(len(df)) # =3500 Tweets currently
print(len(df['author_id'].unique())) # =3046 users currently

# to-do: 
# pull corresponding author information from separate api endpoint for all unique author ids

3501
3046


In [9]:
df.head()

,author_id,context_annotations,conversation_id,created_at,edit_history_tweet_ids,entities,id,in_reply_to_user_id,lang,public_metrics,referenced_tweets,reply_settings,source,text,geo
0,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865785513332738,2020-11-07T00:09:58.000Z,[1324866646834606081],"{'annotations': [{'end': 101, 'normalized_text...",1324866646834606081,1236085967666192385,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866614014210050', 'type': 'replie...",everyone,Twitter Web App,@JavoeGator @JonahLandergan The voting machine...,NaN
1,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865385448042496,2020-11-07T00:09:32.000Z,[1324866538059558913],"{'annotations': [{'end': 100, 'normalized_text...",1324866538059558913,1270762579774902272,en,"{'like_count': 1, 'quote_count': 0, 'reply_cou...","[{'id': '1324866484536012800', 'type': 'replie...",everyone,Twitter Web App,@hannahyusuck @saintherons The voting machines...,NaN
2,1229060372105134080,[{'domain': {'description': 'Real world events...,1324866302100508676,2020-11-07T00:08:55.000Z,[1324866382966775810],"{'annotations': [{'end': 88, 'normalized_text'...",1324866382966775810,1661961012,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866302100508676', 'type': 'replie...",everyone,Twitter Web App,@FoxFyrePhotos The voting machines related to ...,NaN
3,1229060372105134080,[{'domain': {'description': 'Real world events...,1324865813325680641,2020-11-07T00:08:40.000Z,[1324866319896940544],"{'annotations': [{'end': 85, 'normalized_text'...",1324866319896940544,1314213370434744326,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324865813325680641', 'type': 'replie...",everyone,Twitter Web App,@bertzbeess The voting machines related to the...,NaN
4,1229060372105134080,[{'domain': {'description': 'Named people in t...,1324835859812798465,2020-11-07T00:08:27.000Z,[1324866264913899522],"{'annotations': [{'end': 98, 'normalized_text'...",1324866264913899522,20939466,en,"{'like_count': 0, 'quote_count': 0, 'reply_cou...","[{'id': '1324866011049435137', 'type': 'replie...",everyone,Twitter Web App,@rajesh06 @NateSilver538 The voting machines r...,NaN


In [11]:
# Create list of author IDs
author_id_array = df['author_id'].to_numpy()

In [38]:
bearer_token = "AAAAAAAAAAAAAAAAAAAAAGyajQEAAAAAZ3zJK19R33SFv7N9lPOOAP4gzsE%3DM61aIZYEZBsVQjdvkti9BmV0jZBfPyKj8ojsFCoqkopcbAFXjO"

In [34]:
def create_url(next_token: str, iteration: int, ids):
    #user_fields = "user.fields=created_at,description,entities,id,location,name,public_metrics,username"
    start_time = 'start_time=2020-11-04T00:00:00.000Z'
    end_time = 'end_time=2020-11-14T00:00:00.000Z'
    url = "https://api.twitter.com/2/users?ids={}".format(ids)
    return url

In [15]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2TweetLookupPython"
    return r


In [16]:
def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

In [41]:
ids = [int(val) for val in author_id_array]

# open file in write mode
with open(r'authors.txt', 'w') as fp:
    for item in ids:
        fp.write("%s\n" % item)
    print('Done')

Done


In [40]:
ids = [int(val) for val in author_id_array]
count = 0
for i in range(1, 3500, 10):
    ids = ids[i:i+10]
    ids = str(ids).replace(" ", "")[1:-1]
    url = create_url(None, count, ids)
    print(url)
    json_response = connect_to_endpoint(url)
    json_object = js.dumps(json_response, indent=4, sort_keys=True)

    filename = f"response{count}_user.json"
    with open(filename, "w") as outfile:
        outfile.write(json_object)
    count+=1


https://api.twitter.com/2/users?ids=1229060372105134080,1229060372105134080,1229060372105134080,1229060372105134080,1229060372105134080,1229060372105134080,1229060372105134080,1229060372105134080,916308444025638913,916308444025638913
200
https://api.twitter.com/2/users?ids=5134080,
400


Exception: Request returned an error: 400 {"errors":[{"parameters":{"ids":["5134080,"]},"message":"The `ids` query parameter value [] is not valid"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}

## Numerical Analysis  